In [2]:
import numpy as np

### Amplifiers:

Each amplifier has a gain (Gi) and a temperature noise (T_Ni) (which can be calculated from the noise figure (dB)).
In the following dictionary each key is a different stage of the output line starting from the chip.
Each key has a list in which the first value is the gain of the amplifier (Gi) and the second one the temperature noise of
the amplifier (you can check the temperature noise or the noise factor in the datasheets).

The chip also has a temperature noise! This temperature noise is calculated with the jupyter notebook for the input line with
attenuators.

In [3]:
def temp_noise(NF):
    # Use this function when the datasheet species the noise factor instead the temperature noise.
    return 300*(10**(NF/10)-1)

Amp = {'Chip': [0, 0.030], '4K':[30,temp_noise(0.1)], 'RT1':[35, 57], 'RT2': [30, 57]}

Now, we use the recursive expression

$T_{N,i} = T_{N,i-1} + \frac{T_{amp, i}}{\prod_{j=1}^{i-1}G_j}$

Where $T_{N,i}$ is the noise temperature at the output of the $i$ amplifier, $T_{amp,i}$ is the temperature noise of this amplifier, and $G_j$ is the gain of each previous amplifier.

This means that the noise introduced by the $i$ amplifier is supressed by all of the previous amplifiers. For example, if we have three amplifiers in the output line:

$T_{final} = T_{chip} + T_{amp,1} + \frac{T_{amp,2}}{G_1} + \frac{T_{amp,3}}{G_1 \cdot G_2}$

with:

$G_i = 10^{\frac{amp(dB)}{10}}$.

In [19]:
G = [1.0]
for key in Amp.keys():
    G.append(10**(Amp[key][0]/10))

T_N = []
j = 1
for key in Amp.keys():      
    T_N.append(Amp[key][1]/np.prod(G[:j]))
    T_noise = np.sum(T_N[:j])
    message = 'At the output of %s stage the temperature noise is %.3f K'%(key, T_noise) 
    print(message)
    j+=1

message = 'Output temperature noise is %.3f K'%(T_noise)
print('\033[1m' + message + '\033[0m')

At the output of Chip stage the temperature noise is 0.030 K
At the output of 4K stage the temperature noise is 7.018 K
At the output of RT1 stage the temperature noise is 7.075 K
At the output of RT2 stage the temperature noise is 7.075 K
Output temperature noise is 7.075 K


### Conclussion:

You can check that the temperature noise at the output is decided by the first amplifier in the chain. The amplifier with lowest noise figure (lowest temperature noise) must be the closest one to the chip. Don't forget that this amplifier also must have the highest gain as possible to supress the noise comming from the second amplifier.

Also, we can calculate the power noise per unit of bandwith $\left( \frac{dBm}{Hz} \right)$ comming from the output line:

In [24]:
kB = 1.38065e-23 # J/K
Power_noise = 10*np.log10(kB*T_noise)
print('\033[1m' + 'The power noise is %.2f dBm/Hz'%(Power_noise) + '\033[0m')

The power noise is -220.10 dBm/Hz


You can use this value to compare the noise floor of your system with the noise floor of the equipment. In this way you know which part of the setup is limiting the experiment.